### Anaylse results

For each market ID look to see if the market was a success or not. Using the odds stored at the time of saving the dataframe see if there was a net profit for different risk levels.

In [2]:
import json
import pandas as pd
import bz2

In [ ]:
import requests
  
application_key = #Application Key
 
payload = 'INSERT CREDENTIALS'
headers = {'X-Application': 'test', 'Content-Type': 'application/x-www-form-urlencoded'}
 
resp = requests.post('https://identitysso-cert.betfair.com/api/certlogin', data=payload, cert=('Betfair.crt', 'client-2048.pem'), headers=headers)
 
resp_json = resp.json()
session_token = resp_json['sessionToken']

if resp.status_code == 200:
    print(resp_json['loginStatus'])
    print(session_token)

else:
    print("Request failed.")

In [3]:
def get_odds(market_id):
    '''Return odds of all runners in a market'''
    
    odds_req = [{
        "jsonrpc": "2.0",
        "method": "SportsAPING/v1.0/listMarketCatalogue",
        "params": {
            "marketIds": [market_id],
            "priceProjection": {
                "priceData": ["EX_BEST_OFFERS", "EX_TRADED"],
                "virtualise": "true"
            }
        },
        "id": 1
    }]
    
    application_key = # Application key

    headers = {'X-Application': application_key, 
               'X-Authentication': session_token,
               'Accept': 'application/json'
               }


    url="https://api.betfair.com/exchange/betting/json-rpc/v1"

    jsonrpc_req = json.dumps(odds_req[0])

    odds_response = requests.post(url, data=jsonrpc_req, headers=headers)
    
    odds_json = json.loads(odds_response.text)
    
    return odds_json['result'][0]

In [4]:
def get_result(market_id):
    '''Return odds of all runners in a market'''
    
    odds_req = [{
        "jsonrpc": "2.0",
        "method": "SportsAPING/v1.0/listMarketBook",
        "params": {
            "marketIds": [market_id],
            "marketStatus": ["CLOSED"]
            # "priceProjection": {
            #     "priceData": ["EX_BEST_OFFERS", "EX_TRADED"],
            #     "virtualise": "true",
                
            },
        #},
        "id": 1
    }
    ]
    
    application_key = # Application key

    headers = {'X-Application': application_key, 
               'X-Authentication': session_token,
               'Accept': 'application/json'
               }


    url="https://api.betfair.com/exchange/betting/json-rpc/v1"

    jsonrpc_req = json.dumps(odds_req[0])

    odds_response = requests.post(url, data=jsonrpc_req, headers=headers)
    
    odds_json = json.loads(odds_response.text)
    
    return odds_json

In [5]:
def get_market(event_id):
    '''Return odds of all runners in a market'''
    
    market_req = [
        {
            "jsonrpc": "2.0",
            "method": "SportsAPING/v1.0/listMarketCatalogue",
            "params": {
                "filter": {
                    "eventIds": [event_id],
                },
                "maxResults": "200",
                "marketProjection": [
                    "COMPETITION",
                    "EVENT",
                    "EVENT_TYPE",
                    "RUNNER_DESCRIPTION",
                    "RUNNER_METADATA",
                    "MARKET_START_TIME"
                ]
            },
            "id": 1
        }
    ]
    
    application_key = # Application key

    headers = {'X-Application': application_key, 
               'X-Authentication': session_token,
               'Accept': 'application/json'
               }


    url="https://api.betfair.com/exchange/betting/json-rpc/v1"

    jsonrpc_req = json.dumps(market_req[0])

    market_response = requests.post(url, data=jsonrpc_req, headers=headers)
    
    market_json = json.loads(market_response.text)
    
    return market_json['result']

In [11]:
num_markets = len(market_response)
runners = []

for i in range(0, num_markets):
    market_id = market_response[i]['marketId']
    market_name = market_response[i]['marketName']
    
    num_runners = len(market_response[i]['runners']) 
    market_odds = get_odds(market_id)
   
    for j in range(0, num_runners):
        runnerName = market_response[i]['runners'][j]['runnerName']
        runnerId = market_response[i]['runners'][j]['selectionId']
        handicap = market_odds['runners'][j]['handicap']
        result = market_odds['runners'][j]['status']
        
        runner = {'marketId': market_id, 'marketName': market_name, 'runnerName': runnerName, 'handicap': handicap, 'runnerId': runnerId, 'result': result}
        runners.append(runner)

In [12]:
results_df = pd.DataFrame(runners)
results_df

""


### Load pre match odds

In [4]:
pre_match = pd.read_csv('Games/ManCity-vs-RealMadrid-Runners.csv')
pre_match

,marketId,marketName,runnerName,handicap,runnerId,runnerOdds,risk
0,1.213153,Half Time,Real Madrid,0.0,2426,3.80,Too high
1,1.213153,Half Time,Man City,0.0,47999,2.88,Too high
2,1.213153,Half Time,The Draw,0.0,58805,2.40,Too high
3,1.213153,Over/Under 4.5 Goals,Under 4.5 Goals,0.0,1222347,1.19,Low
4,1.213153,Over/Under 4.5 Goals,Over 4.5 Goals,0.0,1222346,5.60,Too high
...,...,...,...,...,...,...,...
1005,1.213153,Player Scorecast,Rodri 3 - 2,0.0,6696407,1000.00,Too high
1006,1.213153,Player Scorecast,Rodri 1 - 1,0.0,6696408,130.00,Too high
1007,1.213153,Player Scorecast,Rodri 2 - 2,0.0,6696409,200.00,Too high
1008,1.213153,Player Scorecast,Rodri 3 - 3,0.0,6696410,1000.00,Too high


### Results

In [3]:
results = pd.read_csv('Games/ManCity-vs-RealMadrid-Results.csv')
results

,Unnamed: 0,marketId,marketName,runnerName,handicap,runnerId,runnerStatus
0,0,1.213153,PENALTY_TAKEN,Yes,0.0,30246,LOSER
1,1,1.213153,PENALTY_TAKEN,No,0.0,110503,WINNER
2,2,1.213153,OVER_UNDER_15,Under 1.5 Goals,0.0,1221385,LOSER
3,3,1.213153,OVER_UNDER_15,Over 1.5 Goals,0.0,1221386,WINNER
4,4,1.213153,BOTH_TEAMS_TO_SCORE,Yes,0.0,30246,WINNER
...,...,...,...,...,...,...,...
123,123,1.213153,TEAM_B_1,Real Madrid -1,0.0,6582710,LOSER
124,124,1.213153,DRAW_NO_BET,Real Madrid,0.0,2426,REMOVED
125,125,1.213153,DRAW_NO_BET,Man City,0.0,47999,REMOVED
126,126,1.213153,ALT_TOTAL_GOALS,Under,0.5,7044483,LOSER


### Analysis

In [10]:
low_runners = pre_match[pre_match['risk'] == 'Low']['runnerId']
low_runners

3      1222347
5      1485567
63     2542448
77     5851483
90       47972
        ...   
688     526010
689     526015
690    3647908
691    4316861
723    1221386
Name: runnerId, Length: 100, dtype: int64

In [16]:
low_risk = pre_match[pre_match['risk'] == 'Low']

In [25]:
combo = pd.merge(low_risk, results, how = 'left', on = 'runnerId')
combo

,marketId_x,marketName_x,runnerName_x,handicap_x,runnerId,runnerOdds,risk,Unnamed: 0,marketId_y,marketName_y,runnerName_y,handicap_y,runnerStatus
0,1.213153,Over/Under 4.5 Goals,Under 4.5 Goals,0.0,1222347,1.19,Low,66.0,1.213153,OVER_UNDER_45,Under 4.5 Goals,0.0,WINNER
1,1.213153,Over/Under 5.5 Goals,Under 5.5 Goals,0.0,1485567,1.06,Low,56.0,1.213153,OVER_UNDER_55,Under 5.5 Goals,0.0,WINNER
2,1.213153,Over/Under 6.5 Goals,Under 6.5 Goals,0.0,2542448,1.01,Low,60.0,1.213153,OVER_UNDER_65,Under 6.5 Goals,0.0,WINNER
3,1.213153,Over/Under 0.5 Goals,Over 0.5 Goals,0.0,5851483,1.05,Low,13.0,1.213153,FIRST_HALF_GOALS_05,Over 0.5 Goals,0.0,WINNER
4,1.213153,Over/Under 0.5 Goals,Over 0.5 Goals,0.0,5851483,1.05,Low,19.0,1.213153,OVER_UNDER_05,Over 0.5 Goals,0.0,WINNER
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,1.213153,Correct Score 2 Home,Any unquoted,0.0,4316861,1.01,Low,NaN,NaN,NaN,NaN,NaN,NaN
193,1.213153,Over/Under 1.5 Goals,Over 1.5 Goals,0.0,1221386,1.25,Low,3.0,1.213153,OVER_UNDER_15,Over 1.5 Goals,0.0,WINNER
194,1.213153,Over/Under 1.5 Goals,Over 1.5 Goals,0.0,1221386,1.25,Low,39.0,1.213153,TEAM_A_OVER_UNDER_15,Over 1.5 Goals,0.0,LOSER
195,1.213153,Over/Under 1.5 Goals,Over 1.5 Goals,0.0,1221386,1.25,Low,81.0,1.213153,FIRST_HALF_GOALS_15,Over 1.5 Goals,0.0,LOSER


In [57]:
combo = combo.dropna(axis = 0)
odds_results = pd.DataFrame(combo['runnerOdds'])
odds_results['status'] = combo['runnerStatus']
odds_results = odds_results.reset_index()

In [58]:
odds_results

,index,runnerOdds,status
0,0,1.19,WINNER
1,1,1.06,WINNER
2,2,1.01,WINNER
3,3,1.05,WINNER
4,4,1.05,WINNER
...,...,...,...
159,182,1.02,LOSER
160,193,1.25,WINNER
161,194,1.25,LOSER
162,195,1.25,LOSER


### Calculate theoretical net profit

Assume £1 bets on all bets

In [61]:
winnings = 0
losses = 0
for i in range(0, len(odds_results)):
    if odds_results['status'][i] == 'WINNER':
        prize = odds_results['runnerOdds'][i] - 1
        winnings += prize
    elif odds_results['status'][i] == 'LOSER':
        loss = 1
        losses += 1

In [63]:
print('Total winnings:', winnings)
print('Total losses:', losses)
print('Profit:', winnings - losses)

Total winnings: 9.800000000000002
Total losses: 67
Profit: -57.199999999999996


Note: not feasible to only place bets on low risk markets.

Next step: Create model to identify which bets are low risk, rather than using market prices